<a href="https://colab.research.google.com/github/gongx030/etv2_pioneer/blob/master/scRNA_seq_D1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of scRNA-seq of Etv2 induced reprogramming at D0 and D1

by Wuming Gong



Explore the diversified MEF population and see which subpopulation differentiated into HE population

In [ ]:
start_time0 <- Sys.time()

### Install external packages

In [ ]:
system('apt-get -q install libgsl-dev', intern = TRUE)

[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "  libgsl23 libgslcblas0"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 [6] "Suggested packages:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
 [7] "  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html"                                                           

In [ ]:
system('apt-get install libcairo2-dev', intern = TRUE) # required by R package ComplexHeatmap 

[1] "Reading package lists..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "Building dependency tree..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 [3] "Reading state information..."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [4] "The following additional packages will be installed:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [5] "  libcairo-script-interpreter2 libpixman-1-dev libxcb-shm0-dev"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [6] "Suggested packages:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
 [7] "  libcairo2-doc"                                                                                                       

In [ ]:
system('apt-get install libudunits2-dev') # required by R package 'units'

In [ ]:
system('apt-get install -y libgdal-dev libgeos-dev libproj-dev') # required by R package sf

### Install R packages

In [ ]:
install.packages('BiocManager')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
required_packages <- c(
  'devtools',
  'ggplot2', 'circlize', 'ComplexHeatmap',
  'TxDb.Mmusculus.UCSC.mm10.knownGene',
  'BiocGenerics', 'DelayedArray', 'DelayedMatrixStats', 'limma', 'S4Vectors', 'SingleCellExperiment', 'SummarizedExperiment', 'batchelor', 'Matrix.utils', 'units', 'sf' # required by Monocle3
  , 'org.Mm.eg.db', 'CHIPseeker', 'clusterProfiler', 'ggplot2', 'tidyr', 
  'biomaRt', 'GO.db','biomaRt', 'ReactomePA', 'ggnewscale',
  'GO.db' 
)

In [ ]:
missing_packages <- required_packages[!required_packages %in% rownames(installed.packages())]
if (length(missing_packages) > 0){
    BiocManager::install(missing_packages)
}

Bioconductor version 3.12 (BiocManager 1.30.10), R 4.0.3 (2020-10-10)

Installing package(s) 'BiocVersion', 'circlize', 'ComplexHeatmap',
  'TxDb.Mmusculus.UCSC.mm10.knownGene', 'BiocGenerics', 'DelayedArray',
  'DelayedMatrixStats', 'limma', 'S4Vectors', 'SingleCellExperiment',
  'SummarizedExperiment', 'batchelor', 'Matrix.utils', 'units', 'sf',
  'org.Mm.eg.db', 'CHIPseeker', 'clusterProfiler', 'biomaRt', 'GO.db',
  'ReactomePA', 'ggnewscale'

Warning message:
“package ‘CHIPseeker’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message:
“Perhaps you meant ‘ChIPseeker’ ?”
also installing the dependencies ‘bit’, ‘Rhtslib’, ‘formatR’, ‘bit64’, ‘plogr’, ‘bitops’, ‘zlibbioc’, ‘Rsamtools’, ‘GenomicAlignments’, ‘rhdf5filters’, ‘lambda.r’, ‘futile.options’, ‘data.table’, ‘gridExtra’, ‘fastmatch’, ‘tweenr’, ‘polycli

### Install Monocle3 from github

In [ ]:
devtools::install_github('cole-trapnell-lab/leidenbase', force = TRUE)

In [ ]:
devtools::install_github('cole-trapnell-lab/monocle3', force = TRUE)

### Load R packages

In [ ]:
library(devtools)

In [ ]:
library(SummarizedExperiment)
library(RColorBrewer)
library(monocle3)
library(dplyr)
library(ggplot2)
library(Matrix)
library(circlize)
library(igraph)
library(ComplexHeatmap)
library(org.Mm.eg.db)
library(TxDb.Mmusculus.UCSC.mm10.knownGene)
library(clusterProfiler)
library(ggplot2)
library(tidyr)
library(biomaRt)
library(Matrix)
library(GO.db)
library(matrixStats)
library(ComplexHeatmap)
library(stringr)
library(DOSE)
library(ReactomePA)
library(ggnewscale)
library(GOSemSim)
library(enrichplot)

## Load processed scRNA-seq data

In [ ]:
remote_rds_file <- 'https://s3.msi.umn.edu/gongx030/etv2_pioneer/data/processed_Etv2_scRNAseq.rds'
system(sprintf('wget -x -c -nH %s', remote_rds_file))

In [ ]:
local_rds_file <- gsub('https://s3.msi.umn.edu/', './', remote_rds_file)
se <- readRDS(local_rds_file)
table(colData(se)$group)

## Subset the cells from MEF and D1

In [ ]:
se <- se[, colData(se)$group %in% c('MEF_NoDox', 'MEF_Dox_D1')]

In [ ]:
table(colData(se)$group)

## Trajectory analysis

In [ ]:
num_pca_dim <- 50L
min_expr_cells <- 10L
k_neighbors <- 200L

In [ ]:
counts <- assays(se)$counts
cell_metadata <- colData(se)
colnames(counts) <- 1:ncol(counts)
rownames(cell_metadata) <- 1:ncol(counts)
gene_metadata <- rowData(se)
rownames(counts) <- gene_metadata[, 'name']
rownames(gene_metadata) <- gene_metadata[, 'name']
gene_metadata$gene_short_name <- gene_metadata[, 'name']

In [ ]:
x <- new_cell_data_set(counts, cell_metadata = cell_metadata, gene_metadata = gene_metadata)

In [ ]:
x <- x[rowSums(assays(x)$counts > 0) >= min_expr_cells, ]

In [ ]:
x <- preprocess_cds(x, method = 'PCA', num_dim = num_pca_dim)

In [ ]:
x <- reduce_dimension(x, reduction_method = 'UMAP', verbose = TRUE, cores = 2L)

In [ ]:
x <- cluster_cells(x, reduction_method = 'UMAP', k = k_neighbors)

In [ ]:
x <- learn_graph(x, use_partition = FALSE)

In [ ]:
x

## Plot the trajectories

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
plot_cells(
  x, 
  label_groups_by_cluster = FALSE,  
  color_cells_by = "group", 
  group_label_size = 4,
  show_trajectory_graph = TRUE,
  label_cell_groups = FALSE,
  label_roots = FALSE,
  label_leaves = FALSE,
  label_branch_points = FALSE,
  trajectory_graph_segment_size = 0.5,
  cell_size = 1,
) + scale_color_manual(
    values = c(
      "MEF_NoDox" = "lightgreen", 
      "MEF_Dox_D1" = "lightblue"
    ),
    name = 'group'
  ) +
  theme_bw() +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())

## Determine the center vertices of each cell type

In [ ]:
groups <- unique(colData(x)$group)
CT <- sparseMatrix(
  i = 1:ncol(x),
  j = as.numeric(factor(colData(x)$group, groups)),
  dims = c(ncol(x), length(groups)),
  dimnames = list(NULL, groups)
) # cell id ~ cell type

In [ ]:
CV <- sparseMatrix(
  i = 1:ncol(x),
  j = x@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex,
  dims = c(ncol(x), max(x@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex))
) # cell ~ vertex

In [ ]:
VT <- as(t(CV), 'dgCMatrix') %*% CT # vertex ~ cell type

In [ ]:
C1_center <- order(VT[, 'MEF_Dox_D1'], decreasing = TRUE)[1]
x <- order_cells(x, root_pr_nodes = sprintf('Y_%d', C1_center))
options(repr.plot.width = 8, repr.plot.height = 6)
plot_cells(
  x,
  color_cells_by = "pseudotime",
  label_cell_groups=FALSE,
  label_leaves=FALSE,
  label_branch_points=FALSE,
  graph_label_size=1.5
)

In [ ]:
C2_center <- order(VT[, 'MEF_Dox_D1'], decreasing = TRUE)[2]
x <- order_cells(x, root_pr_nodes = sprintf('Y_%d', C2_center))
options(repr.plot.width = 8, repr.plot.height = 6)
plot_cells(
  x,
  color_cells_by = "pseudotime",
  label_cell_groups=FALSE,
  label_leaves=FALSE,
  label_branch_points=FALSE,
  graph_label_size=1.5
)

## Find the shortest pathway between cell types

In [ ]:
graph <- x@principal_graph_aux[["UMAP"]]$stree %>% 
  graph.adjacency() # graph of vertices only

In [ ]:
path <- shortest_paths(graph, from = C1_center, to = C2_center)$vpath[[1]] %>%
  as.numeric()

Remove the vertices that have no associated cells

In [ ]:
path <- path[colSums(CV[, path] ) > 0]

## Calculate the aggregated normalized counts for each vertex along the path

In [ ]:
x_path <- log(assays(x)$counts + 1) %*% CV[, path] %*% Diagonal(x = 1 / (colSums(CV[, path]) +1 ))

## Find the genes that are significantly up-regulated in each of three trajectories

In [ ]:
res <- do.call('rbind', lapply(1:nrow(x_path), function(i){
  mod <- lm(
    y ~ pseudotime, 
    data.frame(
      y = x_path[i, ],
      pseudotime = 1:ncol(x_path)
    )
  )
  coef(summary(mod))[2, c(1, 4)]
}))

In [ ]:
rownames(res) <- rownames(x_path)

In [ ]:
res_top <- res %>% 
  as.data.frame() %>%
  filter(Estimate > 0) %>%
  arrange(`Pr(>|t|)`) %>% 
  head(10)
res_top

In [ ]:
res_bottom <- res %>% 
  as.data.frame() %>%
  filter(Estimate < 0) %>%
  arrange(`Pr(>|t|)`) %>% 
  head(10)
res_bottom

In [ ]:
y <- x_path[c(rownames(res_top), rownames(res_bottom), 'Kdr'), ] %>% 
  as.matrix() %>% 
  t() %>%
  scale() %>%
  t()

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)
col_fun <- colorRamp2(c(-2, 0, 2), c("purple", "black", "yellow"))
column_annotation <-  columnAnnotation(
  group = colData(x)$group[CV[, path] %>% t() %>% max.col()],
  col = list(
    group = c(
      "MEF_NoDox" = "lightgreen", 
      "MEF_Dox_D1" = "lightblue"
    )
  ),
  show_legend = TRUE,
  annotation_name_side = 'left'
)
Heatmap(
  y,
  cluster_rows = TRUE,
  cluster_columns = FALSE,
  show_column_dend = FALSE,
  show_row_dend = FALSE,
  show_column_names = FALSE,
  show_row_names = TRUE,
  col = col_fun,
  row_names_gp = gpar(fontsize = 20),
  top_annotation = column_annotation
) 

## Pathway analysis

In [ ]:
res_top_pathway <- res %>% 
  as.data.frame() %>%
  filter(Estimate > 0) %>%
  arrange(`Pr(>|t|)`) %>% 
  head(500)
head(res_top_pathway) #Top 100 genes upregulated using the res_top filter

In [ ]:
res_bottom_pathway <- res %>% 
  as.data.frame() %>%
  filter(Estimate < 0) %>%
  arrange(`Pr(>|t|)`) %>% 
  head(500)
head(res_bottom_pathway) ##Top 100 genes upregulated using the res_bottom filter

,Estimate,Pr(>|t|)
,<dbl>,<dbl>
Ylpm1,-0.02955021,6.458971e-15
Prrc2c,-0.07403798,1.057186e-14
Tmem245,-0.03952889,3.510028e-14
Phip,-0.03582109,4.242656e-14
Ly6e,-0.05623937,5.423112e-14
Sypl,-0.04267287,6.627477e-14


### Convert Gene Symbols to Entrez ID's

In [ ]:
symbols_top <- rownames(res_top_pathway)
symbols_top ##gene symbols from res_top

[1] "Psmc5"         "Etfb"          "Cops6"         "Cope"         
  [5] "Acy1"          "Prmt1"         "Acot10"        "Fdps"         
  [9] "Nthl1"         "Psmc4"         "Upp1"          "Acadvl"       
 [13] "Xlr"           "Uqcrc1"        "Eno3"          "Hspb1"        
 [17] "Lgals3"        "Yjefn3"        "Hspb2"         "Renbp"        
 [21] "Psmc2"         "Capg"          "Nnat"          "Myl7"         
 [25] "Isyna1"        "Aimp2"         "Naprt"         "Eci1"         
 [29] "Rpp40"         "Psmb6"         "Lce1g"         "Samm50"       
 [33] "Rogdi"         "Bloc1s1"       "Crabp1"        "Nup62cl"      
 [37] "Psmb5"         "Slc6a15"       "3830403N18Rik" "Ndufa9"       
 [41] "Ruvbl2"        "Lce1h"         "Ndufv1"        "Fam71f2"      
 [45] "Fgf13"         "Trim54"        "Dnajc17"       "Serpinb2"     
 [49] "Fahd2a"        "Nudt17"        "Fcf1"          "Dpysl3"       
 [53] "Dynap"         "Eno1"          "Phb"           "Aldoa"        
 [57] "Fam241b"       "Xlr3a"         "Nefl"          "Hoxb5os"      
 [61] "Acot13"        "Rasgrf1"       "Chaf1b"        "Gsta1"        
 [65] "Iqcd"          "Asl"           "Ngef"          "Gzmd"         
 [69] "Xlr4a"         "Mecr"          "Arl2"          "Asns"         
 [73] "Ubb"           "Eif3g"         "Spink2"        "Gm39473"      
 [77] "Gcat"          "Hsd17b10"      "Npl"           "3300005D01Rik"
 [81] "Tbcb"          "Ddx39"         "A730056A06Rik" "Rbm42"        
 [85] "Rars"          "Atp6v0e2"      "Cct5"          "Krt19"        
 [89] "1700003F12Rik" "Gm12415"       "Eno1b"         "Rnh1"         
 [93] "Taldo1"        "Ubl7"          "Apip"          "Igfbp6"       
 [97] "Rbks"          "Psmd4"         "1700012B09Rik" "Eif2s3y"

In [ ]:
symbols_bottom <- rownames(res_bottom_pathway)
symbols_bottom ##gene symbols from res_bottom

[1] "Ylpm1"         "Prrc2c"        "Tmem245"       "Phip"         
  [5] "Ly6e"          "Sypl"          "Ankrd11"       "Gclc"         
  [9] "Zfhx4"         "Slc39a10"      "Taf1d"         "Ahctf1"       
 [13] "Acaca"         "Gtf3c2"        "Nav3"          "Ddost"        
 [17] "Prpf38b"       "Cpd"           "Mtdh"          "Ppm1a"        
 [21] "Eif2s3x"       "Wnk1"          "Elovl6"        "Eif4g1"       
 [25] "Zfp106"        "Heatr5a"       "Alyref"        "Gxylt1"       
 [29] "Tm4sf1"        "Tspan4"        "Zc3h4"         "Hnrnph1"      
 [33] "Prrc2b"        "Txndc5"        "Suz12"         "Eea1"         
 [37] "Cpsf2"         "Pam"           "Atp11c"        "Ccdc47"       
 [41] "Slc12a2"       "Bptf"          "Lztfl1"        "Macf1"        
 [45] "Mycbp2"        "Pabpc4"        "Man1a2"        "Ankrd26"      
 [49] "Smc1a"         "Matr3"         "Larp1"         "Asph"         
 [53] "Baz1b"         "Cep170"        "Amfr"          "Ttc14"        
 [57] "Mlec"          "Zcchc11"       "Dhx36"         "Lpgat1"       
 [61] "Huwe1"         "Hnrnpa0"       "Cnot6l"        "2610507B11Rik"
 [65] "Arid1a"        "Ppic"          "Rcc2"          "Arfgef1"      
 [69] "Sacs"          "Khsrp"         "Tmed9"         "Lpl"          
 [73] "Gpatch8"       "Purb"          "Gja1"          "Tmem30a"      
 [77] "Sel1l"         "Taf15"         "Uxs1"          "Yy1"          
 [81] "Rapgef6"       "Bcap29"        "Tpbg"          "Ktn1"         
 [85] "Sfpq"          "Tra2a"         "Acsl5"         "Ipo7"         
 [89] "Phf14"         "Polr2a"        "Eif4ebp2"      "Impad1"       
 [93] "Rsbn1l"        "Evl"           "Gpd2"          "Ptp4a2"       
 [97] "Usp16"         "Abcb1b"        "Cdv3"          "Akap1"

In [ ]:
top_id <- mapIds(org.Mm.eg.db, symbols_top, 'ENTREZID', 'SYMBOL') ##Converting symbols to geneID

'select()' returned 1:1 mapping between keys and columns



In [ ]:
bottom_id <- mapIds(org.Mm.eg.db, symbols_bottom, 'ENTREZID', 'SYMBOL')##Converting symbols to geneID

'select()' returned 1:1 mapping between keys and columns



### GO Enrichment analysis 

In [ ]:
top_dat <- enrichGO(gene = top_id,
                OrgDb         = org.Mm.eg.db,
                ont           = "BP",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.05,
                qvalueCutoff  = 0.05,
        readable      = TRUE) #res_top over representation data

In [ ]:
top_dat

#
# over-representation test
#
#...@organism 	 Mus musculus 
#...@ontology 	 BP 
#...@keytype 	 ENTREZID 
#...@gene 	 chr [1:100] "19184" "110826" "26893" "59042" "109652" "15469" "64833" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...28 enriched terms found
'data.frame':	28 obs. of  9 variables:
 $ ID         : chr  "GO:0045899" "GO:0045898" "GO:0016054" "GO:0046395" ...
 $ Description: chr  "positive regulation of RNA polymerase II transcription preinitiation complex assembly" "regulation of RNA polymerase II transcription preinitiation complex assembly" "organic acid catabolic process" "carboxylic acid catabolic process" ...
 $ GeneRatio  : chr  "3/91" "3/91" "7/91" "7/91" ...
 $ BgRatio    : chr  "12/23328" "14/23328" "223/23328" "223/23328" ...
 $ pvalue     : num  1.23e-05 2.03e-05 2.72e-05 2.72e-05 3.64e-05 ...
 $ p.adjust   : num  0.00823 0.00823 0.00823 0.00823 0.0088 ...
 $ qvalue     : num  0.00727 0.00727 0.00727 0.00727 0.00777 ...
 $ geneID     : chr  "Psmc5/Psmc

In [ ]:
top_dat$Description

[1] "positive regulation of RNA polymerase II transcription preinitiation complex assembly"
 [2] "regulation of RNA polymerase II transcription preinitiation complex assembly"         
 [3] "organic acid catabolic process"                                                       
 [4] "carboxylic acid catabolic process"                                                    
 [5] "small molecule catabolic process"                                                     
 [6] "RNA polymerase II preinitiation complex assembly"                                     
 [7] "generation of precursor metabolites and energy"                                       
 [8] "transcription preinitiation complex assembly"                                         
 [9] "nucleotide metabolic process"                                                         
[10] "nucleoside phosphate metabolic process"                                               
[11] "positive regulation of transcription initiation from RNA polymerase II promoter"      
[12] "aspartate family amino acid metabolic process"                                        
[13] "cellular amino acid metabolic process"                                                
[14] "glycolytic process"                                                                   
[15] "ATP metabolic process"                                                                
[16] "ATP generation from ADP"                                                              
[17] "regulation of transcription initiation from RNA polymerase II promoter"               
[18] "ADP metabolic process"                                                                
[19] "positive regulation of DNA-templated transcription, initiation"                       
[20] "purine nucleoside diphosphate metabolic process"                                      
[21] "purine ribonucleoside diphosphate metabolic process"                                  
[22] "nucleoside diphosphate phosphorylation"                                               
[23] "N-acetylneuraminate metabolic process"                                                
[24] "fatty acid beta-oxidation using acyl-CoA dehydrogenase"                               
[25] "ribonucleoside diphosphate metabolic process"                                         
[26] "nucleotide phosphorylation"                                                           
[27] "anterograde axonal transport"                                                         
[28] "regulation of DNA-templated transcription, initiation"

In [ ]:
top_subset <- top_dat %>% filter(!str_detect(Description, 'negative|positive')) ##subsetting data to exclude terms that have negative and positive

In [ ]:
bottom_dat <- enrichGO(gene = bottom_id,
                OrgDb         = org.Mm.eg.db,
                ont           = "BP",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.05,
                qvalueCutoff  = 0.05,
        readable      = TRUE) #res_bottom over representation data

In [ ]:
bottom_dat

#
# over-representation test
#
#...@organism 	 Mus musculus 
#...@ontology 	 BP 
#...@keytype 	 ENTREZID 
#...@gene 	 chr [1:99] "56531" "226562" "242474" "83946" "17069" "19027" "77087" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...24 enriched terms found
'data.frame':	24 obs. of  9 variables:
 $ ID         : chr  "GO:0043488" "GO:0043487" "GO:0061013" "GO:1903311" ...
 $ Description: chr  "regulation of mRNA stability" "regulation of RNA stability" "regulation of mRNA catabolic process" "regulation of mRNA metabolic process" ...
 $ GeneRatio  : chr  "7/94" "7/94" "7/94" "8/94" ...
 $ BgRatio    : chr  "104/23328" "116/23328" "128/23328" "266/23328" ...
 $ pvalue     : num  2.13e-07 4.49e-07 8.74e-07 1.23e-05 1.35e-05 ...
 $ p.adjust   : num  0.00037 0.00039 0.000507 0.004694 0.004694 ...
 $ qvalue     : num  0.000328 0.000346 0.000449 0.004162 0.004162 ...
 $ geneID     : chr  "Pabpc4/Larp1/Dhx36/Hnrnpa0/Cnot6l/Khsrp/Taf15" "Pabpc4/Larp1/Dhx36/Hnrnpa0/Cnot6l/Khsrp/Taf15" "P

In [ ]:
bottom_dat$Description

[1] "regulation of mRNA stability"                           
 [2] "regulation of RNA stability"                            
 [3] "regulation of mRNA catabolic process"                   
 [4] "regulation of mRNA metabolic process"                   
 [5] "regulation of translation"                              
 [6] "negative regulation of translation"                     
 [7] "mRNA catabolic process"                                 
 [8] "regulation of cellular amide metabolic process"         
 [9] "negative regulation of cellular amide metabolic process"
[10] "RNA catabolic process"                                  
[11] "RNA stabilization"                                      
[12] "RNA localization"                                       
[13] "mRNA processing"                                        
[14] "positive regulation of catabolic process"               
[15] "negative regulation of RNA catabolic process"           
[16] "RNA splicing"                                           
[17] "positive regulation of translation"                     
[18] "positive regulation of cellular catabolic process"      
[19] "negative regulation of GTPase activity"                 
[20] "mRNA destabilization"                                   
[21] "RNA destabilization"                                    
[22] "positive regulation of cellular amide metabolic process"
[23] "nucleobase-containing compound catabolic process"       
[24] "positive regulation of mRNA metabolic process"

In [ ]:
bottom_subset <- bottom_dat %>% filter(!str_detect(Description, 'negative|positive')) ####subsetting data to exclude terms that have negative and positive

### KEGG pathway analysis

In [ ]:
kegg_top <- enrichKEGG(gene = top_id,
                organism = 'mouse',
                keyType = 'kegg',
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.05,
                qvalueCutoff  = 0.05)

Reading KEGG annotation online:


Reading KEGG annotation online:




In [ ]:
kegg_top

#
# over-representation test
#
#...@organism 	 mmu 
#...@ontology 	 KEGG 
#...@keytype 	 kegg 
#...@gene 	 chr [1:100] "19184" "110826" "26893" "59042" "109652" "15469" "64833" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...16 enriched terms found
'data.frame':	16 obs. of  9 variables:
 $ ID         : chr  "mmu01230" "mmu03050" "mmu05012" "mmu05022" ...
 $ Description: chr  "Biosynthesis of amino acids" "Proteasome" "Parkinson disease" "Pathways of neurodegeneration - multiple diseases" ...
 $ GeneRatio  : chr  "8/48" "6/48" "10/48" "12/48" ...
 $ BgRatio    : chr  "79/8907" "47/8907" "247/8907" "472/8907" ...
 $ pvalue     : num  7.56e-09 1.61e-07 5.80e-07 5.17e-06 1.06e-05 ...
 $ p.adjust   : num  5.60e-07 5.96e-06 1.43e-05 9.57e-05 1.57e-04 ...
 $ qvalue     : num  4.14e-07 4.41e-06 1.06e-05 7.08e-05 1.16e-04 ...
 $ geneID     : chr  "109652/13808/13806/11674/109900/27053/433182/21351" "19184/23996/19181/19175/19173/19185" "19184/23996/22273/19181/19175/19173/66108/17995/22

In [ ]:
kegg_top$Description

[1] "Biosynthesis of amino acids"                      
 [2] "Proteasome"                                       
 [3] "Parkinson disease"                                
 [4] "Pathways of neurodegeneration - multiple diseases"
 [5] "Prion disease"                                    
 [6] "Alzheimer disease"                                
 [7] "Amyotrophic lateral sclerosis"                    
 [8] "Huntington disease"                               
 [9] "Spinocerebellar ataxia"                           
[10] "Glycolysis / Gluconeogenesis"                     
[11] "Carbon metabolism"                                
[12] "Pentose phosphate pathway"                        
[13] "HIF-1 signaling pathway"                          
[14] "Arginine biosynthesis"                            
[15] "Oxidative phosphorylation"                        
[16] "RNA degradation"

In [ ]:
kegg_bottom <- enrichKEGG(gene = bottom_id,
                organism = 'mouse',
                keyType = 'kegg',
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.05,
                qvalueCutoff  = 0.05)

In [ ]:
kegg_bottom

#
# over-representation test
#
#...@organism 	 mmu 
#...@ontology 	 KEGG 
#...@keytype 	 kegg 
#...@gene 	 chr [1:99] "56531" "226562" "242474" "83946" "17069" "19027" "77087" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...4 enriched terms found
'data.frame':	4 obs. of  9 variables:
 $ ID         : chr  "mmu04141" "mmu03013" "mmu01212" "mmu00061"
 $ Description: chr  "Protein processing in endoplasmic reticulum" "RNA transport" "Fatty acid metabolism" "Fatty acid biosynthesis"
 $ GeneRatio  : chr  "5/39" "5/39" "3/39" "2/39"
 $ BgRatio    : chr  "172/8907" "183/8907" "62/8907" "19/8907"
 $ pvalue     : num  0.000857 0.001132 0.002456 0.003048
 $ p.adjust   : num  0.0334 0.0334 0.045 0.045
 $ qvalue     : num  0.0292 0.0292 0.0393 0.0393
 $ geneID     : chr  "13200/105245/17156/23802/20338" "26905/208643/21681/230721/13688" "107476/170439/433256" "107476/433256"
 $ Count      : int  5 5 3 2
#...Citation
  Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
  clusterProfiler

In [ ]:
kegg_bottom$Description

[1] "Protein processing in endoplasmic reticulum"
[2] "RNA transport"                              
[3] "Fatty acid metabolism"                      
[4] "Fatty acid biosynthesis"

### Reactome Pathway analysis

In [ ]:
reactome_top <- enrichPathway(gene = top_id,
                organism = "mouse",
                pvalueCutoff  = 0.05)

In [ ]:
reactome_top

#
# over-representation test
#
#...@organism 	 mouse 
#...@ontology 	 Reactome 
#...@keytype 	 ENTREZID 
#...@gene 	 chr [1:100] "19184" "110826" "26893" "59042" "109652" "15469" "64833" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...115 enriched terms found
'data.frame':	115 obs. of  9 variables:
 $ ID         : chr  "R-MMU-450408" "R-MMU-5687128" "R-MMU-75815" "R-MMU-349425" ...
 $ Description: chr  "AUF1 (hnRNP D0) binds and destabilizes mRNA" "MAPK6/MAPK4 signaling" "Ubiquitin-dependent degradation of Cyclin D" "Autodegradation of the E3 ubiquitin ligase COP1" ...
 $ GeneRatio  : chr  "8/55" "8/55" "7/55" "7/55" ...
 $ BgRatio    : chr  "56/8733" "73/8733" "51/8733" "52/8733" ...
 $ pvalue     : num  1.64e-09 1.43e-08 2.48e-08 2.85e-08 2.85e-08 ...
 $ p.adjust   : num  6.31e-07 1.17e-06 1.17e-06 1.17e-06 1.17e-06 ...
 $ qvalue     : num  4.56e-07 8.49e-07 8.49e-07 8.49e-07 8.49e-07 ...
 $ geneID     : chr  "19184/23996/15507/19181/19175/19173/22187/19185" "19184/23996/1550

In [ ]:
reactome_top$Description

[1] "AUF1 (hnRNP D0) binds and destabilizes mRNA"                                                                        
  [2] "MAPK6/MAPK4 signaling"                                                                                              
  [3] "Ubiquitin-dependent degradation of Cyclin D"                                                                        
  [4] "Autodegradation of the E3 ubiquitin ligase COP1"                                                                    
  [5] "Ubiquitin Mediated Degradation of Phosphorylated Cdc25A"                                                            
  [6] "p53-Independent DNA Damage Response"                                                                                
  [7] "p53-Independent G1/S DNA damage checkpoint"                                                                         
  [8] "Regulation of RUNX2 expression and activity"                                                                        
  [9] "Regulation of RUNX3 expression and activity"                                                                        
 [10] "FBXL7 down-regulates AURKA during mitotic entry and in early mitosis"                                               
 [11] "Degradation of AXIN"                                                                                                
 [12] "Regulation of mRNA stability by proteins that bind AU-rich elements"                                                
 [13] "Degradation of GLI1 by the proteasome"                                                                              
 [14] "Stabilization of p53"                                                                                               
 [15] "Degradation of DVL"                                                                                                 
 [16] "Dectin-1 mediated noncanonical NF-kB signaling"                                                                     
 [17] "GLI3 is processed to GLI3R by the proteasome"                                                                       
 [18] "NIK-->noncanonical NF-kB signaling"                                                                                 
 [19] "CDT1 association with the CDC6:ORC:origin complex"                                                                  
 [20] "SCF(Skp2)-mediated degradation of p27/p21"                                                                          
 [21] "Asymmetric localization of PCP proteins"                                                                            
 [22] "Hedgehog ligand biogenesis"                                                                                         
 [23] "Activation of NF-kappaB in B cells"                                                                                 
 [24] "Transcriptional regulation by RUNX2"                                                                                
 [25] "Autodegradation of Cdh1 by Cdh1:APC/C"                                                                              
 [26] "p53-Dependent G1 DNA Damage Response"                                                                               
 [27] "p53-Dependent G1/S DNA damage checkpoint"                                                                           
 [28] "MAPK family signaling cascades"                                                                                     
 [29] "Oxygen-dependent proline hydroxylation of Hypoxia-inducible Factor Alpha"                                           
 [30] "RUNX1 regulates transcription of genes involved in differentiation of HSCs"                                         
 [31] "Regulation of RAS by GAPs"                                                                                          
 [32] "Assembly of the pre-replicative complex"                                                                            
 [33] "G1/S DNA Damage Checkpoints

In [ ]:
reactome_bottom <- enrichPathway(gene = bottom_id,
                organism = "mouse",
                pvalueCutoff  = 0.05)

In [ ]:
reactome_bottom ##No pathway terms found with 0.05 as the p-value cutoff

#
# over-representation test
#
#...@organism 	 mouse 
#...@ontology 	 Reactome 
#...@keytype 	 ENTREZID 
#...@gene 	 chr [1:99] "56531" "226562" "242474" "83946" "17069" "19027" "77087" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...0 enriched terms found
#...Citation
  Guangchuang Yu, Qing-Yu He. ReactomePA: an R/Bioconductor package for
  reactome pathway analysis and visualization. Molecular BioSystems
  2016, 12(2):477-479 
